# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

### Импорт данных

**Признаки**
* `DateCrawled` — дата скачивания анкеты из базы
* `VehicleType` — тип автомобильного кузова
* `RegistrationYear` — год регистрации автомобиля
* `Gearbox` — тип коробки передач
* `Power` — мощность (л. с.)
* `Model` — модель автомобиля
* `Kilometer` — пробег (км)
* `RegistrationMonth` — месяц регистрации автомобиля
* `FuelType` — тип топлива
* `Brand` — марка автомобиля
* `Repaired` — была машина в ремонте или нет
* `DateCreated` — дата создания анкеты
* `NumberOfPictures` — количество фотографий автомобиля
* `PostalCode` — почтовый индекс владельца анкеты (пользователя)
* `LastSeen` — дата последней активности пользователя

**Целевой признак**

`Price` — цена (евро)

In [46]:
# импорт библиотек
import pandas as pd
import numpy as np

In [47]:
# импорт данных
data = pd.read_csv('./autos.csv')

display(data.info())
display(data.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

None

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
5,2016-04-04 17:36:23,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,2016-04-04 00:00:00,0,33775,2016-04-06 19:17:07
6,2016-04-01 20:48:51,2200,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,no,2016-04-01 00:00:00,0,67112,2016-04-05 18:18:39
7,2016-03-21 18:54:38,0,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no,2016-03-21 00:00:00,0,19348,2016-03-25 16:47:58
8,2016-04-04 23:42:13,14500,bus,2014,manual,125,c_max,30000,8,petrol,ford,NaN,2016-04-04 00:00:00,0,94505,2016-04-04 23:42:13
9,2016-03-17 10:53:50,999,small,1998,manual,101,golf,150000,0,NaN,volkswagen,NaN,2016-03-17 00:00:00,0,27472,2016-03-31 17:17:06


In [48]:
# удалим полные дубликаты
data = data.drop_duplicates()
display(len(data))

354365

Смущает цена 0 у некоторых автомобилей, удалим эти строки, поскольку они неинформативны

In [49]:
print(len(data[data['Price'] == 0]))
data = data[data['Price'] != 0].reset_index(drop=True)

10772


Столбцы `RegistrationMonth`, `NumberOfPictures`, `PostalCode`, `DateCrawled` и `LastSeen` принимаем избыточными и спокойно удаляем

In [50]:
data = data.drop(['RegistrationMonth', 'NumberOfPictures', 'PostalCode', 'DateCrawled', 'LastSeen', 'DateCreated'], axis=1)

### Работа с пропусками

Наблюдаем пропуски в столбцах `VehicleType`, `Gearbox`, `Model`, `FuelType` и `Repaired`, пропуски в столбце `Model` восстановить не представляется возможным

In [51]:
model_isna_count = len(data[data['Model'].isna()]['Model'])
print('Количество пропусков в столбце Model:', model_isna_count)
print('Доля от общего количества автомобилей:', round(model_isna_count / len(data), 3))

Количество пропусков в столбце Model: 17521
Доля от общего количества автомобилей: 0.051


Удалив пропуски, мы потеряем 5% данных

In [52]:
data = data.dropna(subset=['Model']).reset_index(drop=True)

В столбцах `VehicleType`, `Gearbox` и `FuelType` можно заполнить пропуски, ориентируясь на модель

In [53]:
# создадим датафрейм для записи характеристик каждой модели
models_descriptions = pd.DataFrame(columns=['Brand', 'Model', 'VehicleType', 'Gearbox', 'FuelType'])

for brand in data['Brand'].unique():
    for model in data[data['Brand'] == brand]['Model'].unique():
        records = data.query('Model == @model and Brand == @brand')

        # запишем стандартные параметры этой модели
        vehicle_type = records['VehicleType'].mode()[0]
        gearbox = records['Gearbox'].mode()[0]
        fuel_type = records['FuelType'].mode()[0]
        models_descriptions.loc[len(models_descriptions)] = [brand, model, vehicle_type, gearbox, fuel_type]

display(models_descriptions.head())

,Brand,Model,VehicleType,Gearbox,FuelType
0,volkswagen,golf,sedan,manual,petrol
1,volkswagen,passat,wagon,manual,gasoline
2,volkswagen,polo,small,manual,petrol
3,volkswagen,scirocco,coupe,manual,petrol
4,volkswagen,transporter,bus,manual,gasoline


In [54]:
data = data.drop(['VehicleType', 'Gearbox', 'FuelType'], axis=1)

# последовательно применим к каждому столбцу
data = data.merge(models_descriptions, on=['Brand', 'Model'], how='left')

display(data.info())
display(data.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326072 entries, 0 to 326071
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             326072 non-null  int64 
 1   RegistrationYear  326072 non-null  int64 
 2   Power             326072 non-null  int64 
 3   Model             326072 non-null  object
 4   Kilometer         326072 non-null  int64 
 5   Brand             326072 non-null  object
 6   Repaired          267779 non-null  object
 7   VehicleType       326072 non-null  object
 8   Gearbox           326072 non-null  object
 9   FuelType          326072 non-null  object
dtypes: int64(4), object(6)
memory usage: 24.9+ MB


None

,Price,RegistrationYear,Power,Model,Kilometer,Brand,Repaired,VehicleType,Gearbox,FuelType
0,480,1993,0,golf,150000,volkswagen,NaN,sedan,manual,petrol
1,9800,2004,163,grand,125000,jeep,NaN,suv,auto,gasoline
2,1500,2001,75,golf,150000,volkswagen,no,sedan,manual,petrol
3,3600,2008,69,fabia,90000,skoda,no,small,manual,petrol
4,650,1995,102,3er,150000,bmw,yes,sedan,manual,petrol
5,2200,2004,109,2_reihe,150000,peugeot,no,small,manual,petrol
6,14500,2014,125,c_max,30000,ford,NaN,bus,manual,gasoline
7,999,1998,101,golf,150000,volkswagen,NaN,sedan,manual,petrol
8,2000,2004,105,3_reihe,150000,mazda,no,sedan,manual,petrol
9,2799,2005,140,passat,150000,volkswagen,yes,wagon,manual,gasoline


In [55]:
print('Доля пропусков в Repaired:', round(len(data[data['Repaired'].isna()]) / len(data), 3))
print('Процент ремонтированных машин к неремонтируемым:', 
      round(len(data[data['Repaired'] == 'yes']) / len(data), 3), 'к', 
      round(len(data[data['Repaired'] == 'no']) / len(data), 3))

Доля пропусков в Repaired: 0.179
Процент ремонтированных машин к неремонтируемым: 0.098 к 0.723


Почти наверняка пропуски в столбце `Repaired` обусловлены тем, что машина не ремонтировалась, а терять такой большой процент данных (практически 20%) нецелесообразно, а также преобразуем в булев тип

In [56]:
def repaired_into_bool(value):
    if value == 'yes':
        return 1
    return 0


data['Repaired'] = data['Repaired'].apply(repaired_into_bool)

### Предобработка

Для некоторых моделей потребуется перевести категориальные данные в численные:

In [57]:
# применим One Hot Encoding к Gearbox и FuelType
data_encoded = pd.get_dummies(data, columns=['Gearbox', 'FuelType'], drop_first=True, dtype=int)

# к Brand, Model и VehicleType применим y Encoding
for category in ['Model', 'Brand', 'VehicleType']:
    data_encoded[category] = data.groupby(category)['Price'].transform('mean')

display(data_encoded.head())

,Price,RegistrationYear,Power,Model,Kilometer,Brand,Repaired,VehicleType,Gearbox_manual,FuelType_petrol
0,480,1993,0,4299.323727,150000,4618.620556,0,4766.993876,1,1
1,9800,2004,163,6440.798561,125000,7358.716535,0,9132.485274,0,0
2,1500,2001,75,4299.323727,150000,4618.620556,0,4766.993876,1,1
3,3600,2008,69,4224.542233,90000,6041.791447,0,2987.729009,1,1
4,650,1995,102,5135.656134,150000,6471.321851,1,4766.993876,1,1


Разделим данные:

In [58]:
y = data['Price']
X_encoded = data_encoded.drop(['Price'], axis=1)

Стандартизируем данные:

In [59]:
from sklearn.preprocessing import StandardScaler

numeric = ['RegistrationYear', 'Power', 'Model', 'Brand', 'VehicleType']

scaler = StandardScaler()
scaler.fit(X_encoded[numeric])
X_encoded[numeric] = scaler.transform(X_encoded[numeric])

display(X_encoded.head())

,RegistrationYear,Power,Model,Kilometer,Brand,Repaired,VehicleType,Gearbox_manual,FuelType_petrol
0,-0.164028,-0.612045,-0.132843,150000,-0.002903,0,0.104803,1,1
1,0.000749,0.278821,0.746038,125000,1.810135,0,3.282297,0,0
2,-0.044190,-0.202137,-0.132843,150000,-0.002903,0,0.104803,1,1
3,0.060667,-0.234930,-0.163534,90000,0.938766,0,-1.190264,1,1
4,-0.134069,-0.054571,0.210396,150000,1.222973,1,0.104803,1,1


In [60]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=12)

## Обучение моделей

Будем анализировать следующие модели: линейную регрессию, решающее дерево, случайный лес и градиентный бустинг

Для оценки качества моделей будем использовать RMSE в качестве основной метрики, по заданию значение RMSE должно быть меньше 2500

### Линейная регрессия

In [61]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

linear_model = LinearRegression()
print('Время затраченное на обучение:')
%time linear_model.fit(X_train, y_train)
print('\nВремя затраченное на предсказание')
%time linear_model_predictions = linear_model.predict(X_test)
RMSE_linear_model = root_mean_squared_error(linear_model_predictions, y_test)
print('\nRMSE линейной модели:', round(RMSE_linear_model, 3))

Время затраченное на обучение:
CPU times: total: 15.6 ms
Wall time: 54.9 ms

Время затраченное на предсказание
CPU times: total: 0 ns
Wall time: 4.01 ms

RMSE линейной модели: 3423.763


RMSE линейной модели больше допустимого значения, однако скорость ее обучения и для предсказаний довольно высока, попробуем применить кросс-валидацию:

In [62]:
%%time
from sklearn.model_selection import cross_val_score

new_linear_model = LinearRegression()
result = cross_val_score(new_linear_model, X_encoded, 
                         y, cv=5, scoring='neg_mean_squared_error')
RMSE_scores = np.sqrt(-result)
print('Средний RMSE линейной регрессии при кросс-валидации:', 
        round(np.array(RMSE_scores).mean(), 3))
print('\nВремя работы при кросс-валидации:')

Средний RMSE линейной регрессии при кросс-валидации: 3401.862

Время работы при кросс-валидации:
CPU times: total: 281 ms
Wall time: 429 ms


Линейная модель не может дать нам необходимого качества предсказаний

### Решающее дерево

In [63]:
from sklearn.tree import DecisionTreeRegressor

# подбор гиперпараметров
best_depth = 0
best_score = 4000

for depth in range(5, 30): 
    tree_model = DecisionTreeRegressor(random_state=12, max_depth=depth)
    tree_model.fit(X_train, y_train) 
    tree_model_prediction = tree_model.predict(X_test)
    RMSE_tree_model = root_mean_squared_error(tree_model_prediction, y_test) 
    if RMSE_tree_model < best_score:
        best_depth = depth
        best_score = RMSE_tree_model

print('Лучшая глубина:', best_depth)


Лучшая глубина: 14


In [64]:
# обучаем модель с полученной глубиной
tree_model = DecisionTreeRegressor(random_state=12, max_depth=best_depth)
print('Время затраченное на обучение:')
%time tree_model.fit(X_train, y_train) 
print('\nВремя затраченное на предсказание:')
%time tree_model_prediction = tree_model.predict(X_test)
RMSE_tree_model = root_mean_squared_error(tree_model_prediction, y_test) 
print('\nRMSE решающего дерева:', round(RMSE_tree_model, 3))

Время затраченное на обучение:
CPU times: total: 422 ms
Wall time: 611 ms

Время затраченное на предсказание:
CPU times: total: 15.6 ms
Wall time: 13 ms

RMSE решающего дерева: 1841.207


Обучение и применение решающего дерева при заданной глубине занимаем намного больше времени, однако RMSE значительно уменьшилась

### Случайный лес

In [65]:
from sklearn.ensemble import RandomForestRegressor

# подбор гиперпараметров
best_estimators = 0
best_score = 4000

for estimators in range(5, 50, 5):
    forest_model = RandomForestRegressor(random_state=12, max_depth=best_depth, 
                                         n_estimators=estimators)
    forest_model.fit(X_train, y_train) 
    forest_model_prediction = forest_model.predict(X_test)
    RMSE_forest_model = root_mean_squared_error(forest_model_prediction, y_test) 
    if RMSE_forest_model < best_score:
        best_estimators = estimators
        best_score = RMSE_forest_model

print('Лучшее количество деревьев:', best_estimators)

Лучшее количество деревьев: 45


In [ ]:
# обучаем модель с лучшими гиперпараметрами
forest_model = RandomForestRegressor(max_depth=best_depth, random_state=12,
                                     n_estimators=best_estimators)
print('Время затраченное на обучение:')
%time forest_model.fit(X_train, y_train)
print('\nВремя затраченное на предсказание:')
%time forest_model_prediction = forest_model.predict(X_test)
RMSE_forest_model = root_mean_squared_error(forest_model_prediction, y_test)
print('\nRMSE случайного леса:', RMSE_forest_model)

Время затраченное на обучение:
CPU times: total: 13.9 s
Wall time: 24.4 s

Время затраченное на предсказание:
CPU times: total: 250 ms
Wall time: 529 ms

RMSE случайного леса: 1695.4901022596325


Модель случайного леса тратит в 4 раза больше времени, чем модель дерева решений, при этом RMSE не сильно уменьшился

### Градиентный бустинг

In [81]:
import sklearn 
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV

# датасеты для тренировки модели
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

# подбор гиперпараметров
param_grid = {
    'num_leaves': [31, 50, 70],
    'learning_rate': [i/1000 for i in range(10, 100, 5)],
    'n_estimators': [i for i in range(100, 1000, 100)],
    'max_depth': [i for i in range(10, 20)]
}

lgb_model = lgb.LGBMRegressor(boosting_type='gbdt', objective='regression', verbose=-1, 
                              callbacks=[
                                  lgb.early_stopping(stopping_rounds=3)])

# используем случайный поиск
grid_search = RandomizedSearchCV(estimator=lgb_model, param_distributions=param_grid, cv=5)
grid_search.fit(X_train, y_train)

print('Лучшие параметры:', grid_search.best_params_)

Лучшие параметры: {'num_leaves': 70, 'n_estimators': 600, 'max_depth': 18, 'learning_rate': 0.09}


In [83]:
# применим лучшие параметры чтобы засечь время
lgb_model = lgb.LGBMRegressor(boosting_type='gbdt', objective='regression', verbose=-1,
                              **grid_search.best_params_)
print('Время затраченное на обучение:')
%time lgb_model.fit(X_train, y_train)
print('\nВремя затраченное на предсказание:')
%time lgb_model_predictions = lgb_model.predict(X_test)
RMSE_lgb_model = root_mean_squared_error(lgb_model_predictions, y_test)
print('\nRMSE градиентного бустинга:', RMSE_lgb_model)

Время затраченное на обучение:
CPU times: total: 8.34 s
Wall time: 3.61 s

Время затраченное на предсказание:
CPU times: total: 2.25 s
Wall time: 813 ms

RMSE градиентного бустинга: 1629.2675219457192


RMSE относительно случайного леса не сильно измнилась, при этом время на обучение значительно сократилось, но время на предсказание увеличилось

## Анализ моделей

Наилучшее качество предсказаний - у ансамблевых методов, при этом у LGB модели время на обучение ниже, а время на предсказывание больше, чем у случайного леса

Качество предсказаний дерева решений не сильно хуже качества ансамблевых методов, но обеспечивает необходимую величину и значительно быстрее по всем параметрам

При этом, подбор подходящих гиперпараметров так же занимает значительное время, но обеспечивает необходимое качество предсказаний

# Вывод

Таким образом, в зависимости от условий, можно использовать градиентный бустинг в задачах с большим обьемом данных и необходимостью минимизировать ошибку, случайный лес, обеспечивающий такое же качество, в задачах, требующих быстрых предсказаний, и дерево решений в задачах с ограниченными ресурсами и временем, где достаточна полученная точность